# Vanishing Sphere (Reducing radius over time)

In [1]:
//#r "./../../../../../public/src/L4-application/BoSSSpad/bin/Release/net6.0/BoSSSpad.dll"
#r "C:\Users\toprak\Documents\BoSSS-code\public\src\L4-application\BoSSSpad\bin\Release\net6.0\BoSSSpad.dll"
using System;
using System.Threading;
using System.Diagnostics;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Solution;
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.BoSSSpad;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Control;
using BoSSS.Solution.XNSECommon;
using BoSSS.Solution.NSECommon;
//using BoSSS.XNSE_Solver;
using BoSSS.Solution.LoadBalancing;
using BoSSS.Solution.LevelSetTools;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Solution.Utils;
using System.IO;
using BoSSS.Application.XNSE_Solver.PhysicalBasedTestcases;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

In [2]:
// Is used at Jenkins to generate individual names (for output .json)
string dbname = System.Environment.GetEnvironmentVariable("DATABASE_NAME");
string buildname = System.Environment.GetEnvironmentVariable("JOB_NAME");
//defaults
buildname = String.IsNullOrEmpty(buildname)? "sphere" : buildname;
//string thedate = $"{System.DateTime.Today.Day}-{System.DateTime.Today.Month}-{System.DateTime.Today.Year}";
dbname = String.IsNullOrEmpty(dbname)? "CondStudy" : dbname; //Collison´2
string table_name = String.Concat(buildname, "_", dbname);

In [7]:
//ExecutionQueues
var myBatch = ExecutionQueues[2];
string WFlowName = table_name;
BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm.Init(WFlowName,myBatch);
BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();
var myDB = BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm.DefaultDatabase; 
myDB

Project name is set to 'sphere_CondStudy'.


{ Session Count = 0; Grid Count = 0; Path = \\dc3\userspace\toprak\BoSSS-Cluster4\sphere_CondStudy }

In [8]:
// set parameterz
//bool AMR = true;
//bool useLoadBal = true;
int NoOfTimeSteps = 80;
bool Steady = false;
bool IncludeConvection = true;
int[] core_sweep = { 1};
int[] PolyDegS = new int[] {3};
int[] ResArray = new int[] {30};
int SpaceDim = 3;
int AMRLevel = 1;
double _partRad = 0.6;
var Gshape = Shape.Sphere;

In [9]:
double xMax = 1, yMax = 1, zMax = 1;
double xMin = -1, yMin = -1, zMin = -1;


# Grid

In [10]:
List<IGridInfo> Grids = new List<IGridInfo>();
if (SpaceDim ==2){
foreach(var Res in ResArray){

    var _xNodes = GenericBlas.Linspace(xMin, xMax, Res + 1);
    var _yNodes = GenericBlas.Linspace(yMin, yMax, Res + 1);

    GridCommons grd;
    string gname = "SphereMerger2D_1BCs";

    Console.WriteLine("Looking for {0} with {1} cells",gname,(_xNodes.Length-1)*(_yNodes.Length-1));

    var tmp = new List<IGridInfo>();

    foreach(var grid in myDB.Grids){
        try{
            if (String.IsNullOrEmpty(grid.Name))
                continue;
                //grid.Name = "Null";
            Console.WriteLine("Found {0} with {1} cells on the database",grid.Name,grid.NumberOfCells);
            bool IsMatch = grid.Name.Equals(gname)&&grid.NumberOfCells==(_xNodes.Length-1)*(_yNodes.Length-1);
            if(IsMatch) tmp.Add(grid);
        }
        catch(Exception ex) {
            Console.WriteLine(ex.Message);
        }
    }
    Console.WriteLine("Found {0} times grids with the same configuration for the Res={1}",tmp.Count,Res);
    
    if(tmp.Count() >= 1){
        Console.WriteLine("Chosing the first one found: "+tmp.Pick(0).Name);
        Grids.Add(tmp.Pick(0));
        //IGridInfo firstPick = Grids.ToArray()[0];
        //var ImportedGrid = myDB.Controller.DBDriver.LoadGrid(firstPick.ID,myDB);
        //grd = (GridCommons)ImportedGrid;EdgeTagName 
        //Console.WriteLine("Chosen Grid:{0}",ImportedGrid.ToString());
        continue;

    }else{

        grd = Grid2D.Cartesian2DGrid(_xNodes, _yNodes);

    
        grd.Name = gname;

        grd.EdgeTagNames.Add(1, "Pressure_Outlet");
        grd.DefineEdgeTags(delegate (double[] X) {
            return 1;
        });
                
    myDB.SaveGrid(ref grd);
    Grids.Add(grd);
    }
    
}
Console.WriteLine("\nThe grids to be simulated:");
}
Grids


In [11]:
if (SpaceDim ==3){
foreach(var Res in ResArray){

    var _xNodes = GenericBlas.Linspace(xMin, xMax, Res + 1);
    var _yNodes = GenericBlas.Linspace(yMin, yMax, Res + 1);
    var _zNodes = GenericBlas.Linspace(zMin, zMax, Res + 1);

    GridCommons grd;
    string gname = "SphereMerger_3D_1BCs";

    Console.WriteLine("Looking for {0} with {1} cells",gname,(_xNodes.Length-1)*(_yNodes.Length-1)*(_zNodes.Length-1));

    var tmp = new List<IGridInfo>();

    foreach(var grid in myDB.Grids){
        try{
            if (String.IsNullOrEmpty(grid.Name))
                continue;
                //grid.Name = "Null";
            Console.WriteLine("Found {0} with {1} cells on the database",grid.Name,grid.NumberOfCells);
            bool IsMatch = grid.Name.Equals(gname)&&grid.NumberOfCells==(_xNodes.Length-1)*(_yNodes.Length-1)*(_zNodes.Length-1);
            if(IsMatch) tmp.Add(grid);
        }
        catch(Exception ex) {
            Console.WriteLine(ex.Message);
        }
    }
    Console.WriteLine("Found {0} times grids with the same configuration for the Res={1}",tmp.Count,Res);
    
    if(tmp.Count() >= 1){
        Console.WriteLine("Chosing the first one found: "+tmp.Pick(0).Name);
        Grids.Add(tmp.Pick(0));
        //IGridInfo firstPick = Grids.ToArray()[0];
        //var ImportedGrid = myDB.Controller.DBDriver.LoadGrid(firstPick.ID,myDB);
        //grd = (GridCommons)ImportedGrid;EdgeTagName 
        //Console.WriteLine("Chosen Grid:{0}",ImportedGrid.ToString());
        continue;

    }else{

        grd = Grid3D.Cartesian3DGrid(_xNodes, _yNodes,_zNodes);   
        grd.Name = gname;

        grd.EdgeTagNames.Add(1, "Pressure_Outlet");
        grd.DefineEdgeTags(delegate (double[] X) {
            return 1;
        });
                
    myDB.SaveGrid(ref grd);
    Grids.Add(grd);
    }
    
}
Console.WriteLine("\nThe grids to be simulated:");
}
Grids


Looking for SphereMerger_3D_1BCs with 27000 cells
Found 0 times grids with the same configuration for the Res=30
Grid Edge Tags changed.

The grids to be simulated:


#0: { Guid = 7a09c7ed-25de-42ba-a8be-6fc2d97d9ec0; Name = SphereMerger_3D_1BCs; Cell Count = 27000; Dim = 3 }


# Control File

In [8]:
Func<IGridInfo, int,double, string, bool, bool, double, XNSE_Control> GenXNSECtrl = 
delegate(IGridInfo grd, int k, double viscosity, string RotAxis, bool AMR, bool LoadBalance, double AggThrs){
    XNSE_Control C = new XNSE_Control();

    // basic database options
    // ======================
    C.SetDatabase(myDB);
    C.savetodb = true;
    int J  = grd.NumberOfCells;

    //Session Settings
    C.GridGuid = grd.ID;
    C.GridPartType = GridPartType.Hilbert;
    //deactivated to get same mesh for initial steps


    C.ImmediatePlotPeriod = 10;
    C.SuperSampling = 0;    

    // Physical Parameters
    // =================== 
    const double rhoA = 1;
    const double Re = 1000;
    double muA = viscosity;
    
    double partRad = _partRad;
    double d_hyd = 2*partRad;
    double VelocityMax = Re * muA / rhoA / d_hyd;
    double anglev = VelocityMax / partRad;
    double[] pos = new double[SpaceDim];
    double ts = 2*Math.PI/anglev/NoOfTimeSteps; //    double ts = Math.PI/anglev/NoOfTimeSteps/10;
    
    C.PhysicalParameters.IncludeConvection = IncludeConvection;
    C.PhysicalParameters.Material = true;
    C.PhysicalParameters.rho_A = rhoA;
    C.PhysicalParameters.mu_A = muA;
    string rotAxis;
    if (RotAxis == null)
        rotAxis = "y";
    else
        rotAxis = RotAxis;


    C.SessionName = string.Format("J{0}_k{1}_agg{2}_AMR{3}_LB{4}_at{5}_r{6}", J, k,AggThrs,AMR,LoadBalance,rotAxis.ToUpper(),partRad);
    C.SessionName += $"_{SpaceDim}D";
    if(IncludeConvection){
        C.SessionName += "_NSE";
        C.Tags.Add("NSE");
    } else {
        C.SessionName += "_Stokes";
        C.Tags.Add("Stokes");
    }
    C.Tags.Add(SpaceDim + "D");
    if(Steady)C.Tags.Add("steady");
    else C.Tags.Add("transient");


    // DG degrees
    // ==========
    C.SetFieldOptions(k, Math.Max(k, 2));
    C.saveperiod = 5;
    //C.TracingNamespaces = "*";

    C.GridGuid = grd.ID;
    C.GridPartType = GridPartType.Hilbert;
    //C.DynamicLoadbalancing_ClassifierType = ClassifierType.CutCells;
    C.DynamicLoadBalancing_On = LoadBalance;
    C.DynamicLoadBalancing_RedistributeAtStartup = false;
    C.DynamicLoadBalancing_Period = 10;
    C.DynamicLoadBalancing_ImbalanceThreshold = 0.1;
    //C.DynamicLoadBalancing_CellCostEstimatorFactories = BoSSS.Application.XNSE_Solver.Loadbalancing.XNSECellCostEstimator.Factory().ToList();

    // rigid body and boundary conditions
    // ===================
    double rateOfRadius = -1/(double)NoOfTimeSteps/ts;
    C.Rigidbody.SetParameters(pos,anglev,partRad,SpaceDim, rateOfRadius: rateOfRadius);
    C.SessionName += $"_rateOfRadius{rateOfRadius:f2}";
    C.Rigidbody.SpecifyShape(Gshape);
    C.Rigidbody.SetRotationAxis(rotAxis);

    //Console.WriteLine($"tiltAxis.Dim: {tiltAxis.Dim}");
    Console.WriteLine("Angular Velocity: {0}",anglev);
    C.SessionName += $"_W{anglev:f2}";

    C.AddInitialValue(VariableNames.LevelSetCGidx(0), new Formula("X => -1"));
    C.UseImmersedBoundary = true;


    C.AddInitialValue("Pressure", new Formula(@"X => 0"));
    C.AddBoundaryValue("Pressure_Outlet");

    // solver options
    // ===================
    C.CutCellQuadratureType = BoSSS.Foundation.XDG.XQuadFactoryHelper.MomentFittingVariants.Saye;
    C.UseSchurBlockPrec = true;
    C.AgglomerationThreshold = AggThrs;
    C.AdvancedDiscretizationOptions.ViscosityMode = ViscosityMode.FullySymmetric;
    C.Option_LevelSetEvolution2 = LevelSetEvolution.Prescribed;
    C.Option_LevelSetEvolution = LevelSetEvolution.None;
    C.Timestepper_LevelSetHandling = LevelSetHandling.LieSplitting;
    C.LinearSolver = new BoSSS.Solution.AdvancedSolvers.OrthoMGSchwarzConfig() {
        NoOfMultigridLevels = 5,
        ConvergenceCriterion = 1E-3,
        MaxSolverIterations = 100,
        //MaxKrylovDim = 30,
        TargetBlockSize = 10000,
        //verbose = true
    };
    //C.LinearSolver.NoOfMultigridLevels = 5;
    //C.LinearSolver.ConvergenceCriterion = 1E-6;
    //C.LinearSolver.MaxSolverIterations = 200;
    //C.LinearSolver.MaxKrylovDim = 50;
    //C.LinearSolver.TargetBlockSize = 10000;
    //C.LinearSolver.verbose = true;
    C.LinearSolver = LinearSolverCode.exp_Kcycle_schwarz.GetConfig();
    C.NonLinearSolver.SolverCode = NonLinearSolverCode.Newton;
    C.NonLinearSolver.ConvergenceCriterion = 1E-3;
    C.NonLinearSolver.MaxSolverIterations = 50;
    C.NonLinearSolver.verbose = true;

    C.AdaptiveMeshRefinement = AMR;
    if (AMR) {
        C.SetMaximalRefinementLevel(AMRLevel);
        C.AMR_startUpSweeps = AMRLevel;
    }

    // Timestepping
    // ============
    double dt = -1;
    if(Steady){
        C.TimesteppingMode = AppControl._TimesteppingMode.Steady;
        dt = 1000;
        C.NoOfTimesteps = 1;
    } else {
        C.TimesteppingMode = AppControl._TimesteppingMode.Transient;        
        dt = ts;        
        C.NoOfTimesteps = NoOfTimeSteps; //NoOfTimeSteps;
    }
    C.TimeSteppingScheme = TimeSteppingScheme.ImplicitEuler; //BD4
    C.dtFixed = dt;
    C.SkipSolveAndEvaluateResidual = false;
    C.SessionName = "Solver" + !C.SkipSolveAndEvaluateResidual + "_" + C.SessionName;
    var config = new OperatorAnalysisConfig();
    config.CalculateMassMatrix = true;
    config.CalculateGlobalConditionNumbers = false;
    config.CalculateStencilConditionNumbers = true;
    config.PlotStencilConditionNumbers = true;

    C.PostprocessingModules.Add(new BoSSS.Application.XNSE_Solver.Logging.CondLogger(config));
            
    C.SessionName += "_notebookTest"; //1timestep
    //C.PlotAgglomeration = true;
    return C;    
};

In [9]:
var ViscositySweep = new double[]{1E-2};
var axes = new string[]{"z"};
var AMRarray = new bool[] {false};
var LoadBalancearray = new bool[] {false};
var aggThresolds = new double[] {0, 0.1, 0.2, 0.3, 0.4, 0.5}; //{0, 0.1, 0.2, 0.3, 0.4, 0.5};

//var TimeSchemes = new TimeSteppingScheme[] {TimeSteppingScheme.ImplicitEuler, TimeSteppingScheme.BDF4};
List<XNSE_Control> controls = new List<XNSE_Control>();
foreach(IGridInfo grd in Grids)
    foreach(int k in PolyDegS)
            foreach(double v in ViscositySweep)
                foreach(string axis in axes)
                    foreach(bool AMR in AMRarray)
                        foreach(bool LoadBalance in LoadBalancearray)
                            foreach(double agg in aggThresolds)
            {
                controls.Add(GenXNSECtrl(grd,k,v,axis,AMR,LoadBalance,agg));
            }


Console.WriteLine($"Total Number of Control Files: {controls.Count}");
controls.Summary()

Angular Velocity: 13.888888888888891
Angular Velocity: 13.888888888888891
Angular Velocity: 13.888888888888891
Angular Velocity: 13.888888888888891
Angular Velocity: 13.888888888888891
Angular Velocity: 13.888888888888891
Angular Velocity: 13.888888888888891
Angular Velocity: 13.888888888888891
Angular Velocity: 13.888888888888891
Angular Velocity: 13.888888888888891
Angular Velocity: 13.888888888888891
Angular Velocity: 13.888888888888891
Total Number of Control Files: 12


List of 12 instances of XNSE_Control {
  0: BoSSS.Application.XNSE_Solver.XNSE_Control
  1: BoSSS.Application.XNSE_Solver.XNSE_Control
  2: BoSSS.Application.XNSE_Solver.XNSE_Control
  3: BoSSS.Application.XNSE_Solver.XNSE_Control
  4: BoSSS.Application.XNSE_Solver.XNSE_Control
  5: BoSSS.Application.XNSE_Solver.XNSE_Control
  6: BoSSS.Application.XNSE_Solver.XNSE_Control
  7: BoSSS.Application.XNSE_Solver.XNSE_Control
  8: BoSSS.Application.XNSE_Solver.XNSE_Control
  9: BoSSS.Application.XNSE_Solver.XNSE_Control
  10: BoSSS.Application.XNSE_Solver.XNSE_Control
  11: BoSSS.Application.XNSE_Solver.XNSE_Control
}

In [10]:
int k = 0;
foreach(var ctrl in controls){
    string sessname = ctrl.SessionName;
        foreach(int cores in core_sweep){
            ctrl.SessionName = sessname + "_ct"+cores;
                    var aJob   = new Job("Vanishing"+Gshape+ctrl.SessionName,typeof(XNSE));
            //Console.WriteLine(ctrl.Rigidbody.GetTilt());
            aJob.SetControlObject(ctrl);
            aJob.NumberOfMPIProcs         = cores;
            //aJob.ExecutionTime            = "3:00:00";
            aJob.UseComputeNodesExclusive = true;

            //List<string> Cmdtmp = myBatch.AdditionalBatchCommands.ToList();
            //int NoOfNodes = 1;
            //Cmdtmp.Add($"#SBATCH -N {NoOfNodes}");
            //myBatch.AdditionalBatchCommands = Cmdtmp.ToArray();
            //aJob.ExecutionTime = "23:59:00";
            aJob.Activate(myBatch);
            Console.WriteLine("Submitted {0}th \n",k) ;
            k++;           
        }       
    }


Deployments so far (0): ;
Success: 0
job submit count: 0


unable to determine job status - unknown


Deploying job VanishingSphereSolverFalse_J27000_k1_agg0_AMRFalse_LBFalse_atZ_r0.6_3D_NSE_rateOfRadius-2.21_W13.89_condMaMaStenLRfinal_ct8 ... 
Deploying executables and additional files ...
Deployment directory: X:\work\sphere_CondStudy-XNSE_Solver2024Jan12_094039.838704
copied 43 files.
   written file: control.obj
   copied 'linux/amd64-openmpi' runtime.
deployment finished.
Instance #1 of ssh client mt64qocy@lcluster14.hrz.tu-darmstadt.de instantiated.

Submitted 0th 

Deployments so far (0): ;
Success: 0
job submit count: 0


unable to determine job status - unknown


Deploying job VanishingSphereSolverFalse_J27000_k1_agg0.1_AMRFalse_LBFalse_atZ_r0.6_3D_NSE_rateOfRadius-2.21_W13.89_condMaMaStenLRfinal_ct8 ... 
Deploying executables and additional files ...
Deployment directory: X:\work\sphere_CondStudy-XNSE_Solver2024Jan12_094051.472474
copied 43 files.
   written file: control.obj
   copied 'linux/amd64-openmpi' runtime.
deployment finished.

Submitted 1th 

Deployments so far (0): ;
Success: 0
job submit count: 0


...

Deploying job VanishingSphereSolverFalse_J27000_k2_agg0.3_AMRFalse_LBFalse_atZ_r0.6_3D_NSE_rateOfRadius-2.21_W13.89_condMaMaStenLRfinal_ct8 ... 
Deploying executables and additional files ...
Deployment directory: X:\work\sphere_CondStudy-XNSE_Solver2024Jan12_094135.912483
copied 43 files.
   written file: control.obj
   copied 'linux/amd64-openmpi' runtime.
deployment finished.

Submitted 9th 

Deployments so far (0): ;
Success: 0
job submit count: 0


unable to determine job status - unknown


Deploying job VanishingSphereSolverFalse_J27000_k2_agg0.4_AMRFalse_LBFalse_atZ_r0.6_3D_NSE_rateOfRadius-2.21_W13.89_condMaMaStenLRfinal_ct8 ... 
Deploying executables and additional files ...
Deployment directory: X:\work\sphere_CondStudy-XNSE_Solver2024Jan12_094142.185967
copied 43 files.
   written file: control.obj
   copied 'linux/amd64-openmpi' runtime.
deployment finished.

Submitted 10th 

Deployments so far (0): ;
Success: 0
job submit count: 0


unable to determine job status - unknown


Deploying job VanishingSphereSolverFalse_J27000_k2_agg0.5_AMRFalse_LBFalse_atZ_r0.6_3D_NSE_rateOfRadius-2.21_W13.89_condMaMaStenLRfinal_ct8 ... 
Deploying executables and additional files ...
Deployment directory: X:\work\sphere_CondStudy-XNSE_Solver2024Jan12_094149.111812
copied 43 files.
   written file: control.obj
   copied 'linux/amd64-openmpi' runtime.
deployment finished.

Submitted 11th 

